In [4]:
import pybamm

In [5]:
model = pybamm.lithium_ion.SPM()

In [6]:
# Create a new model where the current density is now a variable
# To do so, we replace all instances of the current density in the
# model with a current density variable, which is obtained from the
# FunctionControl submodel
new_model = model.new_copy()
# create the FunctionControl submodel and extract variables
external_circuit_variables = (
    pybamm.external_circuit.FunctionControl(
        model.param, None, model.options, control="algebraic"
    ).get_fundamental_variables()
)

# Perform the replacement
symbol_replacement_map = {
    model.variables[name]: variable
    for name, variable in external_circuit_variables.items()
}
# Don't replace initial conditions, as these should not contain
# Variable objects
replacer = pybamm.SymbolReplacer(
    symbol_replacement_map, process_initial_conditions=False
)
new_model = replacer.process_model(model, inplace=False)

# Update the algebraic equation and initial conditions for
# FunctionControl
# This creates a differential or algebraic equation for the current,
# together with the appropriate guess for the initial condition.
# External circuit submodels are always equations on the current
i_cell = new_model.variables["Current density variable"]
I = new_model.variables["Current [A]"]
I_applied = pybamm.FunctionParameter(
    "Current function [A]", {"Time [s]": pybamm.t * new_model.param.timescale}
)
new_model.algebraic[i_cell] = I - I_applied
new_model.initial_conditions[
    i_cell
] = new_model.param.current_with_time

In [7]:
models = [model, new_model]

In [8]:
sols = []
for model in models:
    sim = pybamm.Simulation(model)
    sol = sim.solve([0, 3600])
    sols.append(sol)

In [9]:
pybamm.dynamic_plot(sols)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [10]:
for sol in sols:
    print(sol.y.shape)

(41, 100)
(42, 100)


In [11]:
pybamm.dynamic_plot(sols[-1], [["Current [A]", "Current density variable"]])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…